In [3]:
import pandas as pd
import pyodbc
import re
from datetime import datetime

In [4]:
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=tcp:txcare.database.windows.net,1433;"
    "DATABASE=Txcare;"
    "UID=fmariscal;"
    "PWD=Pancho49193@2"
)

# Conectar a la base de datos
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Cargar el archivo de Excel (formato .xls)
file_path = r'C:/Users/EJRuiz/Desktop/AMIoil17 KyoeiSG AltaSteel EAF Tamini_67599 MT (GasOnLine) 20230817.xls'
sheet_name = 'RefSh'
df_excel = pd.read_excel(file_path, sheet_name=sheet_name, header=None, usecols="H:BE", skiprows=219, nrows=1)

# Transponer el DataFrame y resetear el índice
df_excel = df_excel.T.reset_index(drop=True)
df_excel.columns = ['ComboKey']

# Obtener los datos de la tabla SQL
query = 'SELECT ComboKey FROM dbo.PlantasGOL'
df_sql = pd.read_sql(query, conn)

# Comparar los datos y encontrar diferencias
excel_set = set(df_excel['ComboKey'].dropna())
sql_set = set(df_sql['ComboKey'].dropna())

# Nombres en Excel pero no en SQL
not_in_sql = excel_set - sql_set

# Nombres en SQL pero no en Excel
not_in_excel = sql_set - excel_set

# Mostrar resultados 
print("Nombres en Excel pero no en SQL:")
print(not_in_sql)

print("Nombres en SQL pero no en Excel:")
print(not_in_excel)

print(len(not_in_sql))
print(len(not_in_excel))
# Cerrar la conexión
cursor.close()
conn.close()


C:\Users\EJRuiz\AppData\Local\Temp\ipykernel_18660\3082833725.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(query, conn)


Nombres en Excel pero no en SQL:
{'NSBS_EAF_66447_MT', 'GLNMTB_SUB_TX2.XXXX_MT', 'GLNMTB_SUB_TX1.xxxx_MT', 'DARAM_HEA_66315_MT', 'GLNSTP_EAF3_NNNN_MT', 'KWT_LMF_17344_MT', 'GLNSTP_EAF4_NNNN_MT', 'GLNTexas_LMF_5116141_MT', 'KWT_EAF_66447_MT', 'OUT_EAF1_66731_LTC', 'GLNCHL_LMF_ZHS79681_MT', 'NSBS_EAF_66716_MT', 'NSBS_EAFb_66716_MT', 'SYS_EAF1_64514_MT', 'GLNSTP_EAF1_NNNN_MT', 'DARAM_HEA_6IH100_MT', 'NSBS_EAF_64789_MT', 'GLNSTP_EAF2_NNNN_MT'}
Nombres en SQL pero no en Excel:
{'GLNMID_EAFA_7002862_MT', 'GLNMID_EAFB_65072_MT', 'GLNCHL_EAF_51015838_MT', 'GLNCHL_EAF_51015838_LTC', 'GLNMID_EAFB_65072_LTC'}
18
5


In [7]:
# Conectar a la base de datos
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()
# Cargar el archivo de Excel (formato .xls)
file_path = r'C:/Users/EJRuiz/Desktop/AMIoil17 KyoeiSG AltaSteel EAF Tamini_67599 MT (GasOnLine) 20230817.xls'
sheet_name = 'RefSh'
ComboKey = pd.read_excel(file_path, sheet_name=sheet_name, header=None, skiprows=219, nrows=1)
FileGol = pd.read_excel(file_path, sheet_name=sheet_name, header=None, skiprows=232, nrows=1)
FurnanceID = pd.read_excel(file_path, sheet_name=sheet_name, header=None, skiprows=242, nrows=1)
Estandar = pd.read_excel(file_path, sheet_name=sheet_name, header=None, skiprows=237, nrows=1)
Ruta = pd.read_excel(file_path, sheet_name=sheet_name, header=None, skiprows=234, nrows=1)
 
 
Headers= ['ComboKey','FileGol','FurnanceID','Estandar','Ruta']

 
    
# Transponer el DataFrame y resetear el índice
ComboKey = ComboKey.T.reset_index(drop=True)
FileGol = FileGol.T.reset_index(drop=True)
FurnanceID = FurnanceID.T.reset_index(drop=True)
Estandar = Estandar.T.reset_index(drop=True)
Ruta = Ruta.T.reset_index(drop=True)
 
lst_inf= [ComboKey,FileGol,FurnanceID,Estandar,Ruta]
 
pd_concat= pd.concat(lst_inf, axis=1)
pd_concat = pd_concat.dropna()
pd_concat = pd_concat[2:-1]
pd_concat.columns= Headers
 
 
#  Compara el final para extraer el numero de tanque
#  Queda comentado la modificacion del nombre nuevo del FileGol
 
tank = []
new_strings = []
for i in pd_concat['FileGol'].values:
    match = re.search(r'_(\d+)_$', i)
    if match:
        tank.append(int(match.group(1)))
        new_strings.append(i)
    else:
        new_strings.append(i + '_0_') 
        tank.append(0)
 
pd_concat['Tanque'] = tank
 
rutas=[]
 
 
for i in pd_concat['Ruta'].values:
    partes = i.split('\\')
    nombre_planta = partes[3]
    if nombre_planta.startswith('Gerdau LN'):
        nombre_planta = 'Gerdau LN'
    elif nombre_planta.startswith('Tata Steel'):
        nombre_planta = 'Tata Steel'
    elif nombre_planta.startswith('Gerdau'):
        nombre_planta = 'Gerdau'
    elif nombre_planta.startswith('Yamato'):
        nombre_planta = 'Yamato'
    elif nombre_planta.startswith('Outokumpu'):
        nombre_planta = 'Outokumpu'
    nueva_ruta = r'O:\Acerias\PlantasSitio\{}\GasOnline'.format(nombre_planta)
    print(nueva_ruta)
    rutas.append(nueva_ruta)
 
pd_concat['Ruta'] = rutas
 
 
hoy = datetime.today()
 
# Formatear la fecha en el formato 'AAAA-DD-M'
fecha_formateada = hoy.strftime('%Y-%d-%m')
 
# Crear una nueva columna 'Fecha' con la fecha formateada
pd_concat['Fecha'] = fecha_formateada
 
# Iterar sobre las filas del DataFrame y ejecutar el procedimiento almacenado para cada fila
for index, row in pd_concat.iterrows():
    cursor.execute("""
        EXEC sp_ActualizaPlantasGOL @accion='GRABAR', @ComboKey=?, @FileGOL=?, @furnaceID=?, @Tank=?, @FechaExportar=?, @Directorio=?
    """, row['ComboKey'], row['FileGol'], row['FurnanceID'], row['Tanque'], row['Fecha'], row['Ruta'])
    
    conn.commit()

# Cerrar la conexión a la base de datos
cursor.close()
conn.close()

O:\Acerias\PlantasSitio\KWT Steel\GasOnline
O:\Acerias\PlantasSitio\KWT Steel\GasOnline
O:\Acerias\PlantasSitio\KWT Steel\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau LN\GasOnline
O:\Acerias\PlantasSitio\Gerdau L